# A Second SkLearn Model

- Based only on the numerical values
- Based on Absolute Values
- No Level
- Target BattleResult, transformed to [-1, 1] Range, 

In [4]:
import sklearn
import sys
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from kedro.extras.datasets.pandas import CSVDataSet

### Loading Data

In [2]:
BattleResults = catalog.load('Battle_Results_Train')

2020-05-23 08:27:20,019 - kedro.io.data_catalog - INFO - Loading data from `Battle_Results_Train` (CSVDataSet)...


In [3]:
BattleResults.head()

,Name_1,Level_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,Name_2,Level_2,HP_2,Attack_2,Defense_2,Sp_Atk_2,Sp_Def_2,Speed_2,Legendary_2,WeatherAndTime,BattleResult
0,Psyduck,8,68,65,59,82,63,69,False,Weedle,75,336,192,172,103,103,279,False,Unknown,-336
1,Rapidash,86,691,707,492,571,571,752,False,Zapdos,72,607,523,499,722,523,577,True,Sunshine,133
2,Dratini,93,636,698,484,538,538,538,False,Marowak,73,446,529,720,328,529,284,False,Windy,324
3,Horsea,6,41,47,85,85,29,72,False,Moltres,86,919,886,797,1088,750,797,True,Night,-919
4,Golbat,78,571,519,449,400,481,567,False,Kingler,96,637,1178,1049,456,456,682,False,Unknown,-560


### Data Pipeline

In [6]:
def RelativeResult(X):
    HP = X.apply(lambda row: row["HP_1"] if row["BattleResult"] > 0 else row["HP_2"], axis=1).values
    relativized = X["BattleResult"].values / HP
    return relativized

battle_labels = RelativeResult(BattleResults)

In [7]:
passthrough_attributes = ['HP_1', 'Attack_1', 'Defense_1', 'Sp_Atk_1', 'Sp_Def_1', 'Speed_1', 
                          'HP_2', 'Attack_2', 'Defense_2', 'Sp_Atk_2', 'Sp_Def_2', 'Speed_2']
full_pipeline = ColumnTransformer([
        ("Pass Through", "passthrough", passthrough_attributes)
    ])
battle_data = full_pipeline.fit_transform(BattleResults); #battle_data

### Train

In [23]:
train_x, val_x, train_y, val_y  = train_test_split(battle_data, battle_labels, test_size=0.2, random_state=42)

In [25]:
train_y

array([-0.65537555,  0.92148148,  1.        , ...,  1.        ,
        0.92490119, -0.7       ])

#### Linear Regression

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [32]:
lin_reg = LinearRegression()
lin_reg.fit(train_x, train_y)
test(lin_reg)

Training RMSE: 0.535644211549331
Validation RMSE: 0.5358601593137552


In [39]:
def test(estimator):
    battle_predictions = estimator.predict(train_x)
    mse = mean_squared_error(battle_predictions, train_y)
    rmse = np.sqrt(mse)
    print("Training RMSE: " + str(rmse))
    battle_predictions = estimator.predict(val_x)
    mse = mean_squared_error(battle_predictions, val_y)
    rmse = np.sqrt(mse)
    print("Validation RMSE: " + str(rmse))

#### SVM

In [34]:
from sklearn.svm import LinearSVR

In [ ]:
svm_reg = LinearSVR(epsilon=0.5, )
svm_reg.fit(train_x, train_y)

In [40]:
test(svm_reg)

Training RMSE: 0.8784365401991504
Validation RMSE: 0.878267800430498


#### Random Forests

#### PCA ?